In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import classifier_utils
from classifier_utils import cnn_model, preprocess_img, get_class
%matplotlib inline
import glob
import os
from skimage import io
from skimage import transform
import keras
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential, Model
import vgg16


## Loading CIFAR10

In [ ]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print (y_train[1]) 

### Array details (dimension, type):

In [ ]:
print (x_train.shape)

In [ ]:
type(x_train)

In [ ]:
print (x_train)

### Image resizing. Vgg16 takes minimum size of 48x48

In [ ]:
x_train_vgg = np.zeros((50000,64,64,3))

for i in range(50000):
    x_train_vgg[i] = transform.resize(x_train[i], (64, 64), order=0)

### Cutting validation set

In [ ]:
size_of_dataset = 50000 #in case of CIFAR 

mask = np.random.rand(size_of_dataset) < 0.8  #array of boolean variables

training_set = x_train_vgg[mask]
training_labels = y_train[mask]

validation_set = x_train_vgg[~mask]
validation_labels = y_train[~mask]

In [ ]:
#1st model
#IMG_SIZE = 32
#2nd model
IMG_SIZE = 64 
NUM_CLASSES = 10

### Reducing amount of pictures

In [ ]:
#2nd model
x_train_vgg = x_train_vgg[0:50000] #bigger value
y_train_vgg = y_train[0:50000]

### Why already calculated values? By default on .resize

In [ ]:
print (x_train_vgg)

### Examples from training set (64x64 and 32x32)

In [ ]:
single_image = x_train_vgg[28:29]
plt.imshow(single_image[0])         

### Zdjecie warstwy: include_top=False

In [ ]:
#first_model = cnn_model(IMG_SIZE,NUM_CLASSES)

In [ ]:
second_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(64,64,3), pooling=max)

### Choose model (remember to set NUM_CLASSES)

In [ ]:
#model = first_model
model = second_model

In [ ]:
model.layers

In [ ]:
model.layers[2]

### Setting all layers to trainable

In [ ]:
for layer in model.layers: layer.trainable=True

In [ ]:
model.summary()

### Alternative way to add last layer and activation 

In [ ]:
#model.add(Dense(NUM_CLASSES, activation='softmax')) .add doesn't work for VGG16

last = model.output

x = Flatten()(last)
preds = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(model.input, preds)

In [ ]:
model.layers

In [ ]:
model.summary()

### Set optimizer for tweaks (leraning rate lr=0.001 to lr=0.0001) !Unquote Adam!

In [ ]:
Adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [ ]:
model.compile(optimizer=Adam,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#NO validation
model.fit(x_train_vgg, y_train_vgg, batch_size=128, epochs=1) 
#with validation
#model.fit(training_set, training_labels, batch_size=128, nb_epoch=1, validation_data=(validation_set, validation_labels))

### Set test size to 64x64

## Evaluation

In [ ]:
x_test_vgg = np.zeros((10000,64,64,3))

for i in range(10000):
    x_test_vgg[i] = transform.resize(x_test[i], (64, 64), order=0)

In [ ]:
print (x_test_vgg.shape)

In [ ]:
model.evaluate(x_test_vgg, y_test)